In [1]:
import os
import numpy
import gzip
from datetime import datetime, timedelta

In [2]:
WORKSPACE_FOLDER = os.path.join(os.path.abspath('..'), 'processing_record')
if not os.path.exists(WORKSPACE_FOLDER):
    os.makedirs(WORKSPACE_FOLDER)
AHI_LST_FOLDER = '/data01/people/beichen/data_fd_et/CEReS_AHILST'

In [3]:
START_TIME = '2018-01-01T00:00:00Z' # local time
END_TIME = '2019-12-31T23:59:59Z'

time_internal = 10  # mins

pixel_size = 0.02 # 0.02°
# FD 180 extent
lst_extent = (85.0, 180.0, -60, 60) # l_lon, r_lon, b_lat, t_lat

# UTC_OFFSET = 9 # hour
# tky_site_loc = (36.146, 137.423) # lat, lon
# crk_site_loc = (38.201, 127.251) # lat, lon
# gck_site_loc = (37.748, 127.162) # lat, lon
# gdk_site_loc = (37.749, 127.149) # lat, lon
# site_names = ['TKY','CRK', 'GCK', 'GDK']
# site_locs = [tky_site_loc, crk_site_loc, gck_site_loc, gdk_site_loc]

In [4]:
site_infos = [# site, lat, lon, UTC+
    ["CLC", 44.59666667, 123.4702778, 8],
    ["JRC", 31.80683056, 119.2172639, 8],
    ["PDC", 26.6, 106.3166667, 8],
    ["PJC", 40.94138889, 121.9602778, 8],
    ["HZF", 51.78111111, 121.0177778, 8],
    ["ZAG", 32.8, 102.55, 8],
    ["DBB", 41.64397222, 110.3314722, 8],
    ["HBG", 37.61666667, 101.3166667, 8],
    ["MPM", 22.4982, 114.0292, 8],
    ["BBY", 43.323, 141.8107, 9],
    ["TKY", 36.14616667, 137.4231111, 9],
    ["CRK", 38.20138889, 127.2505556, 9],
    ["GCK", 37.74833333, 127.1622222, 9],
    ["GDK", 37.74888889, 127.1491667, 9],
    ["SIP", 0.395204167, 102.7645506, 7],
    ["SDP", 0.699491389, 102.793305, 7],
    ["ASM", -22.2828, 133.2493, 9],
    ["CPW", -33.6152, 150.7236, 11],
    ["WAR", -43.095, 146.6545, 11]
]

In [5]:
def read_AHILST_data(ahi_lst_gz, site_infos):
    if os.path.exists(ahi_lst_gz):
        try:
            with gzip.open(ahi_lst_gz, 'rb') as file:
                gz_data = file.read()
                lst_data = numpy.copy(numpy.frombuffer(gz_data, dtype='f4').reshape(6000, 6000)) # 0.02°

                site_array = []
                for site_info in site_infos:
                    site_lat_idx = int((lst_extent[3] - site_info[1])/pixel_size)
                    site_lon_idx = int((site_info[2] - lst_extent[0])/pixel_size)
                    site_v = lst_data[site_lat_idx][site_lon_idx]
                    if site_v == -999:
                        site_v = numpy.NaN
                    site_array.append(site_v)
                return numpy.array(site_array)
        except Exception as e:
            print(ahi_lst_gz)
            print(e)
            nan_array = numpy.zeros(len(site_infos),)
            nan_array[nan_array==0] = numpy.NaN
            return nan_array
    else:
        nan_array = numpy.zeros(len(site_infos),)
        nan_array[nan_array==0] = numpy.NaN
        return nan_array

In [6]:
# 1. record full day from 20171231 to 20191231
# 2. save site data according to localtime with UTC offset
utc_start_date = datetime.strptime(START_TIME, "%Y-%m-%dT%H:%M:%SZ") - timedelta(hours=24)
utc_end_date = datetime.strptime(END_TIME, "%Y-%m-%dT%H:%M:%SZ")

site_record = []
for site_idx in range(len(site_infos)):
    site_record.append([])
temp_date = utc_start_date
while temp_date < utc_end_date:
    current_time_str = temp_date.strftime("%Y%m%d%H%M")
    month_folder = temp_date.strftime("%Y%m")
    day_folder = temp_date.strftime("%Y%m%d")
    file_path = AHI_LST_FOLDER + '/' + month_folder + '/AHILST.v0.' + current_time_str + '.dat.gz'
    site_vs = read_AHILST_data(file_path, site_infos)
    
    for site_idx in range(len(site_infos)):
        UTC_OFFSET = site_infos[site_idx][3]
        this_day_date = temp_date + timedelta(hours=UTC_OFFSET)
        this_day_time = this_day_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        site_record[site_idx].append([this_day_time, str(site_vs[site_idx])])
    
    temp_date = temp_date + timedelta(minutes=time_internal)

In [7]:
for site_idx in range(len(site_infos)):
    site_name = site_infos[site_idx][0]
    UTC_OFFSET = site_infos[site_idx][3]
    site_localrecord = site_record[site_idx][(24*6-UTC_OFFSET*6):-UTC_OFFSET*6]
    site_record_item = numpy.array(site_localrecord)
    numpy.savetxt(os.path.join(WORKSPACE_FOLDER, site_name+'_AHI_LST'+'.csv'), site_record_item, delimiter=",", fmt='%s')